# Hatchet Query Language


This notebook explores [Hatchet](https://github.com/LLNL/hatchet) queries from it's [**Object-based Dialect**](https://hatchet.readthedocs.io/en/latest/query_lang.html) within **Category 1: Quantifier Capabilities**. Below, Fig. 1 provides an overview of all the categories within the Hatcher Query Language.   

|Category ID|Category Description|
|:---------:|:-------------------|
|1          |Quantifier Capabilities|
|2          |String Equivalence and Regex Matching|
|3          |String Containment (contains, starts with, ends with)|
|4          |Basic Numeric Comparison (==, >, >=, etc.)|
|5          |Comparison with Special Values (NaN, Inf, None)|

**Figure 1**: A table of the Hatchet Query Language capabilities, distinguished into categories and their corresponding cateogry ID.

The notebook covers different query types that are capable of matching various number of nodes from [GraphFrame objects](https://hatchet.readthedocs.io/en/latest/user_guide.html). Hatchet offers multiple interfaces to define queries with different trade-offs to verbosity and expressiveness. The entire capability table with information about all the query use cases, categories and capabilities can be found [here](https://docs.google.com/spreadsheets/d/1fKNlHmDJdDbnE4jyMcaFqdnw6ZSaexgm33rOcVAj0do/edit#gid=0).


A hatchet query always finds all **matching paths** for a provided GraphFrame. For this reason, Hatchet queries also takes the shape of a sequence of **predicates** to match, with the added convienience of also allowing to define an accompinying **quantifier** for each predicate. This concept is reflected in Fig. 2, where we interpret queries in terms of filtering or selecting parts of the GraphFrame.

![Graph frames and queries](../common/images/hatchet_query_graphframe.png)

**Figure 2**: A diagram to provide an overview of queries and an example of how queries filter GraphFrames.

***

## Category 1: Quantifier Capabilities

A valid hatchet query requires a **quantifier**. The accepted values for query node quantifiers in the **Object-based Dialect** are:

1. `"."`: Match a single node
2. `"*"`: Match 0 or more nodes
3. `"+"`: Match 1 or more nodes
4. `Integer`: Match an exact number of nodes

### Loading profile data as Hatchet GraphFrame

Hatchet queries are only defined on Hatchet GraphFrames. 
Obtaining a hatchet GraphFrame is straight forward:

1. Import hatchet
2. Use the appropriete reader for the profile/trace at hand

This notebook loads a [Caliper](https://github.com/LLNL/Caliper) profile in JSON format and Caliper is a performance profiling library.

The dataset contains [LULESH (Livermore Unstructured Lagrangian Explicit Shock Hydrodynamics)](https://asc.llnl.gov/codes/proxy-apps/lulesh) performance report data generated by Caliper. LULESH is a highly simplified application designed to solve the Sedov Blast problem, which is a standard hydrodynamics test problem. It performs a hydronomics stencil calculation using both MPI and OpenMP to achieve parallelism. 

This is an interesting profile because it covers a relatively large number of nodes and also spends considerable time in MPI communication routines.

In [1]:
import hatchet as ht
gf = ht.GraphFrame.from_caliper("../../data/lulesh-16nodes/lulesh-annotation-profile-512cores.json")

<IPython.core.display.Javascript object>

### Displaying a Hatchet GraphFrame
A compact overview of a hatchet GraphFrame can be obtained using the `gf.tree()` function. We use this throughout the notebook to display the differences between an original GraphFrame and the resulting GraphFrame after applying a query.

In [2]:
print(gf.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

85405.000 main
├─ 5131.000 MPI_Barrier
├─ 22.000 MPI_Finalize
├─ 60.000 MPI_Irecv
├─ 490.000 MPI_Isend
├─ 63760.000 MPI_Reduce
├─ 227.000 MPI_Wait
├─ 5164.000 MPI_Waitall
└─ 1593.000 lulesh.cycle
   ├─ 920.000 LagrangeLeapFrog
   │  ├─ 4555.000 CalcTimeConstraintsForElems
   │  │  ├─ 143957.000 CalcCourantConstraintForElems
   │  │  └─ 31087.000 CalcHydroConstraintForElems
   │  ├─ 3066.000 LagrangeElements
   │  │  ├─ 122145.000 ApplyMaterialPropertiesForElems
   │  │  │  └─ 3669862.000 EvalEOSForElems
   │  │  │     ├─ 2951673.000 CalcEnergyForElems
   │  │  │     │  └─ 2337222.000 CalcPressureForElems
   │  │  │     └─ 131819.000 CalcSoundSpeedForElems
   │  │  ├─ 404166.000 CalcLagrangeElements
   │  │  │  └─ 4025197.000 CalcKinematicsForElems
   │  │  ├─ 647778.000 CalcQForE

### Displaying a DataFrame
An additional detail perspective can be obtained by viewing the underlying data using a **DataFrame**. Hatchet uses pandas **DataFrames** to store the data on each node of the hierarchy and keeps the graph relationships between the nodes in a different data structure that is kept consistent with the **DataFrame**. The **DataFrame** holds all the numerical and categorical data associated with each node. 

In [8]:
gf.dataframe

time (inc)        time  nid  \
node                                        rank                                
{'name': 'main', 'type': 'region'}          0     53546844.0     85405.0    0   
                                            1     53544897.0     86494.0    0   
                                            2     53546812.0     87314.0    0   
                                            3     53547521.0     87077.0    0   
                                            4     53546355.0     87223.0    0   
...                                                      ...         ...  ...   
{'name': 'MPI_Allreduce', 'type': 'region'} 507    9104187.0   9104187.0   26   
                                            508    8889953.0   8889953.0   26   
                                            509    9599096.0   9599096.0   26   
                                            510   10104611.0  10104611.0   26   
                                            511   13706205.0  13706205.0   26   

                                                           name  
node                                        rank                 
{'name': 'main', 'type': 'region'}          0              main  
                                            1              main  
                                            2              main  
                                            3              main  
                                            4              main  
...                                                         ...  
{'name': 'MPI_Allreduce', 'type': 'region'} 507   MPI_Allreduce  
                                            508   MPI_Allreduce  
                                            509   MPI_Allreduce  
                                            510   MPI_Allreduce  
                                            511   MPI_Allreduce  

[23040 rows x 4 columns]

### Dropping index levels

As a precursor to defining queries, we drop the index level of the GraphFrame using the `drop_index_levels()` Hatchet function. Hatchet hierarchical indexing can be of two types, depending on whether there is a single metric per node or multiple set of metrics per node.  

If a node contains a single metric, the DataFrame will use an `Index` object containing the node column. If a node has additional level of information, Hatchet creates a `MultiIndex` to store the information pertaining to multiple set of metrics per node. `MultiIndex` stores the node column as the "top" level of the index, followed by additional information on the levels below. 

Based on the types of indexing (`Index or MultiIndex`), retrieving data from a DataFrame corresponding to a particular node either retrieves a single or multiple rows. This difference can cause issues when applying query node predicates.
Therefore, it is necessary to get rid of all index levels besides the node column through an aggregation operation on the GraphFrame. Then, a query node predicate can be applied to the GraphFrame. 

In [9]:
gf.drop_index_levels()

### Query type 1: Match a single node

In the simpliest case, one can use this query type to match all single nodes that belong to function calls of a particular library (e.g., MPI). 

The following query matches all single nodes where the predicate that the `name` metric `starts with "MPI_Barrier"` is satisfied:

In [10]:
# single node with names starting with MPI_Barrier
query_1 = [
    (1, {"name": "MPI_Barrier.*"}),
]

The above query, when used with an accepted value for the quantifier, ".", to match all single nodes with a certain predicate, is equivalent to the follwing query:

In [11]:
# single node with names starting with MPI_Barrier
query_1 = [
    (".", {"name": "MPI_Barrier.*"}),
]

The above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame. The `filter()` function takes a user-supplied function or query object and applies that to all rows in the DataFrame. The resulting Series or DataFrame is used to filter the DataFrame to only return rows that are true.

In [12]:
gf_filt = gf.filter(query_1)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [13]:
print(gf_filt.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

4686.619 MPI_Barrier

Legend (Metric: time Min: 4686.62 Max: 4686.62)
█ 4686.62 - 4686.62
█ 4686.62 - 4686.62
█ 4686.62 - 4686.62
█ 4686.62 - 4686.62
█ 4686.62 - 4686.62
█ 4686.62 - 4686.62

name User code    ◀  Only in left graph    ▶  Only in right graph



The query match is also reflected in the DataFrame:

In [14]:
gf_filt.dataframe

,time (inc),time,nid,name
node,,,,
"{'name': 'MPI_Barrier', 'type': 'region'}",4686.619141,4686.619141,39,MPI_Barrier


Then, we use  `drop_index_levels()`  to get rid of all index levels besides the node column in preparation of defining more queries below.

In [15]:
gf.drop_index_levels()

### Query type 2: Match zero or more nodes

In many cases, one may not know how many nodes to match. For this reason hatchet provides the `"*"` and `"+"` literals as a quantifier  `match zero or more nodes` and `match one or more nodes`, respectively.

This query type filters a GraphFrame with the object syntax to find all query paths with zero or more nodes that meet a query predicate. This notebook contains two examples for this query use case. The purpose of the second example is to illustrate the difference between the query use cases that `match zero or more nodes` and `match one or more nodes`.

Note: For matching zero or more nodes, we use the string literal `"*"`. 

**Example 1:**

For the first example, the following query matches all zero or more nodes where the predicate that the `name` metric `starts with "Calc"` is satisfied.

In [28]:
# zero or more nodes with name starting with Calc
query_2_1 = [
    ("*", {"name": "Calc.*"}),
]

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [17]:
gf_filt = gf.filter(query_2_1)

Here, instead of matching only single nodes, entire call stacks can be matched. The resulting GraphFrame now only lists the  nodes that matched the query:

In [18]:
print(gf_filt.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

1561684.193 CalcEnergyForElems
└─ 1226398.809 CalcPressureForElems
446362.916 CalcForceForNodes
└─ 508134.576 CalcVolumeForceForElems
   └─ 13311611.410 CalcHourglassControlForElems
      └─ 4832239.869 CalcFBHourglassForceForElems
400145.020 CalcLagrangeElements
└─ 4029241.518 CalcKinematicsForElems
640901.787 CalcQForElems
├─ 1793326.787 CalcMonotonicQGradientsForElems
└─ 1482072.434 CalcMonotonicQRegionForElems
112708.814 CalcSoundSpeedForElems
4707.689 CalcTimeConstraintsForElems
├─ 157073.449 CalcCourantConstraintForElems
└─ 33242.412 CalcHydroConstraintForElems

Legend (Metric: time Min: 4707.69 Max: 13311611.41)
█ 11980921.04 - 13311611.41
█ 9319540.29 - 11980921.04
█ 6658159.55 - 9319540.29
█ 3996778.81 - 6658159.55
█ 1335398.06 - 3996778.81
█ 4707.69 - 1335398.06

name U

The query match is also reflected in the DataFrame:

In [19]:
gf_filt.dataframe

,time (inc),time,nid,name
node,,,,
"{'name': 'CalcEnergyForElems', 'type': 'region'}",2.788083e+06,1.561684e+06,41,CalcEnergyForElems
"{'name': 'CalcPressureForElems', 'type': 'region'}",1.226399e+06,1.226399e+06,42,CalcPressureForElems
"{'name': 'CalcForceForNodes', 'type': 'region'}",1.909835e+07,4.463629e+05,5,CalcForceForNodes
"{'name': 'CalcVolumeForceForElems', 'type': 'region'}",1.865199e+07,5.081346e+05,7,CalcVolumeForceForElems
"{'name': 'CalcHourglassControlForElems', 'type': 'region'}",1.814385e+07,1.331161e+07,9,CalcHourglassControlForElems
"{'name': 'CalcFBHourglassForceForElems', 'type': 'region'}",4.832240e+06,4.832240e+06,10,CalcFBHourglassForceForElems
"{'name': 'CalcLagrangeElements', 'type': 'region'}",4.429387e+06,4.001450e+05,21,CalcLagrangeElements
"{'name': 'CalcKinematicsForElems', 'type': 'region'}",4.029242e+06,4.029242e+06,22,CalcKinematicsForElems
"{'name': 'CalcQForElems', 'type': 'region'}",3.916301e+06,6.409018e+05,24,CalcQForElems


Then, we use  `drop_index_levels()`  to get rid of all index levels besides the node column in preparation of defining more queries below.

In [20]:
gf.drop_index_levels()

**Example 2:**

In some cases, it is necessary to constrain which nodes to match. In other cases, it may be unknown which functions are called before a particular routine.

For this second example, the first quantifier (`"."`) constrains the filter to single node with the predicate that the metric `name`, `starts with lulesh`. The second quantifier (`"."`) all nodes matching any node, before only `matching zero or more nodes` that satisfy the predicate that the metric `name`, `starts with Calc`.  

In [29]:
# zero or more nodes with several query nodes
query_2_2 = [
    (".", {"name": "lulesh.*"}),
    ("."),
    ("*", {"name": "Calc.*"})
]

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [30]:
gf_filt = gf.filter(query_2_2)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [31]:
print(gf_filt.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

762.863 lulesh.cycle
├─ 958.877 LagrangeLeapFrog
│  └─ 4707.689 CalcTimeConstraintsForElems
│     ├─ 157073.449 CalcCourantConstraintForElems
│     └─ 33242.412 CalcHydroConstraintForElems
└─ 1186.652 TimeIncrement

Legend (Metric: time Min: 762.86 Max: 157073.45)
█ 141442.39 - 157073.45
█ 110180.27 - 141442.39
█ 78918.16 - 110180.27
█ 47656.04 - 78918.16
█ 16393.92 - 47656.04
█ 762.86 - 16393.92

name User code    ◀  Only in left graph    ▶  Only in right graph



The above graph tree demonstrates that when the query to `match zero or more nodes` is executed with the constraints mentioned above, the node with `name "TimeIncrement"` is included, as it satisfies the third quantifier and predicate in the example. This specific node is ommited when the third quantifier in this example is changed to match one or more nodes instead. 

The query match is also reflected in the DataFrame:

In [32]:
gf_filt.dataframe

,time (inc),time,nid,name
node,,,,
"{'name': 'lulesh.cycle', 'type': 'region'}",197931.943359,762.863281,1,lulesh.cycle
"{'name': 'LagrangeLeapFrog', 'type': 'region'}",195982.427734,958.876953,3,LagrangeLeapFrog
"{'name': 'CalcTimeConstraintsForElems', 'type': 'region'}",195023.550781,4707.689453,19,CalcTimeConstraintsForElems
"{'name': 'CalcCourantConstraintForElems', 'type': 'region'}",157073.449219,157073.449219,20,CalcCourantConstraintForElems
"{'name': 'CalcHydroConstraintForElems', 'type': 'region'}",33242.412109,33242.412109,23,CalcHydroConstraintForElems
"{'name': 'TimeIncrement', 'type': 'region'}",1186.652344,1186.652344,2,TimeIncrement


Then, we use  `drop_index_levels()`  to get rid of all index levels besides the node column in preparation of defining more queries below.

In [33]:
gf.drop_index_levels()

### Query type 3: Match one or more nodes

This query type filters a GraphFrame with the object syntax to find all query paths with one or more nodes that meet a query predicate. 

The notebook contains two examples for this query type. The purpose of the second example is to illustrate the difference between the query types that `match zero or more nodes` and `match one or more nodes`.


Note: For matching one or more nodes, we use the string literal `"+"`.

**Example 1:**

For the first example, the metric used for the query is `name` and the predicate is that the `name` metric `starts with "CalcMonotonic"`.

In [34]:
# one or more nodes with name starting with CalcMonotonic
query_3_1 = [
    ("+", {"name": "CalcMonotonic.*"}),
]

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [35]:
gf_filt = gf.filter(query_3_1)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [36]:
print(gf_filt.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

1793326.787 CalcMonotonicQGradientsForElems
1482072.434 CalcMonotonicQRegionForElems

Legend (Metric: time Min: 1482072.43 Max: 1793326.79)
█ 1762201.35 - 1793326.79
█ 1699950.48 - 1762201.35
█ 1637699.61 - 1699950.48
█ 1575448.74 - 1637699.61
█ 1513197.87 - 1575448.74
█ 1482072.43 - 1513197.87

name User code    ◀  Only in left graph    ▶  Only in right graph



The query match is also reflected in the DataFrame:

In [37]:
gf_filt.dataframe

,time (inc),time,nid,name
node,,,,
"{'name': 'CalcMonotonicQGradientsForElems', 'type': 'region'}",1.793327e+06,1.793327e+06,27,CalcMonotonicQGradientsForElems
"{'name': 'CalcMonotonicQRegionForElems', 'type': 'region'}",1.482072e+06,1.482072e+06,33,CalcMonotonicQRegionForElems


Then, we use  `drop_index_levels()`  to get rid of all index levels besides the node column in preparation of defining more queries below.

In [38]:
gf.drop_index_levels()

**Example 2:**

For the second example, we repeat the second example in the previous section but replace the final query node to 'match one or more nodes'. The first quantifier (`"."`) constrains the filter to single node with the predicate that the metric `name`, `starts with lulesh`. The second quantifier (`"."`) all nodes matching any node, before only `matching one or more nodes` that satisfy the predicate that the metric `name`, `starts with Calc`.  

In [39]:
# one or more nodes with several query nodes
query_3_2 = [
    (".", {"name": "lulesh.*"}),
    ("."),
    ("+", {"name": "Calc.*"})
]

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [40]:
gf_filt = gf.filter(query_3_2)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [41]:
print(gf_filt.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

762.863 lulesh.cycle
└─ 958.877 LagrangeLeapFrog
   └─ 4707.689 CalcTimeConstraintsForElems
      ├─ 157073.449 CalcCourantConstraintForElems
      └─ 33242.412 CalcHydroConstraintForElems

Legend (Metric: time Min: 762.86 Max: 157073.45)
█ 141442.39 - 157073.45
█ 110180.27 - 141442.39
█ 78918.16 - 110180.27
█ 47656.04 - 78918.16
█ 16393.92 - 47656.04
█ 762.86 - 16393.92

name User code    ◀  Only in left graph    ▶  Only in right graph



Execution of the second examples for query type 2 (match zero or more nodes) and query type 3 (match one or more nodes) demonstrate the difference between the two query types when used in combination with other quantifiers and predicates. 

The predicate that the metric `name`, `starts with Calc` for this dataset demonstrates that when we `match one or more nodes`, the filter ommits the node with the `name "TimeIncrement"`, as it does not satisfy the third quantifier and predicate in this example. 

This specific node is included when the third quantifier in this example is changed to match zero or more nodes instead. The query match is also reflected in the DataFrame:

In [42]:
gf_filt.dataframe

,time (inc),time,nid,name
node,,,,
"{'name': 'lulesh.cycle', 'type': 'region'}",196745.291016,762.863281,1,lulesh.cycle
"{'name': 'LagrangeLeapFrog', 'type': 'region'}",195982.427734,958.876953,3,LagrangeLeapFrog
"{'name': 'CalcTimeConstraintsForElems', 'type': 'region'}",195023.550781,4707.689453,19,CalcTimeConstraintsForElems
"{'name': 'CalcCourantConstraintForElems', 'type': 'region'}",157073.449219,157073.449219,20,CalcCourantConstraintForElems
"{'name': 'CalcHydroConstraintForElems', 'type': 'region'}",33242.412109,33242.412109,23,CalcHydroConstraintForElems


Then, we use  `drop_index_levels()`  to get rid of all index levels besides the node column in preparation for the final query use case with **Category 1: Quantifier Capabilities**.

In [43]:
gf.drop_index_levels()

### Query type 4: Match exact number of nodes


This query type filters a GraphFrame with the object syntax to find all query paths with an exact number of nodes, provided as an integer, that meets a query predicate.


Note: For matching an exact number of nodes, we use an `"integer value"` to define the number of nodes to match.

The metric used for the query is `name` and the predicate is that the `name` metric `starts with "Calc"`. We have previously applied a query use case to match zero or more nodes that start with the name Calc. However, one can use the following query to concisely match only those nodes that contain `exactly three nodes` that `start with name Calc`. The resulting GraphFrame should be relatively smaller, considering the original GraphFrame and the previous example.

In [44]:
# exactly three nodes with names starting with Calc
query_4 = [
    (3, {"name": "Calc.*"}),
]

Just as before, the above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame.

In [45]:
gf_filt = gf.filter(query_4)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [46]:
print(gf_filt.tree())

    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v2022.1.0

446362.916 CalcForceForNodes
└─ 508134.576 CalcVolumeForceForElems
   └─ 13311611.410 CalcHourglassControlForElems
      └─ 4832239.869 CalcFBHourglassForceForElems

Legend (Metric: time Min: 446362.92 Max: 13311611.41)
█ 12025086.56 - 13311611.41
█ 9452036.86 - 12025086.56
█ 6878987.16 - 9452036.86
█ 4305937.46 - 6878987.16
█ 1732887.77 - 4305937.46
█ 446362.92 - 1732887.77

name User code    ◀  Only in left graph    ▶  Only in right graph



The query match is also reflected in the DataFrame:

In [47]:
gf_filt.dataframe

,time (inc),time,nid,name
node,,,,
"{'name': 'CalcForceForNodes', 'type': 'region'}",1.909835e+07,4.463629e+05,5,CalcForceForNodes
"{'name': 'CalcVolumeForceForElems', 'type': 'region'}",1.865199e+07,5.081346e+05,7,CalcVolumeForceForElems
"{'name': 'CalcHourglassControlForElems', 'type': 'region'}",1.814385e+07,1.331161e+07,9,CalcHourglassControlForElems
"{'name': 'CalcFBHourglassForceForElems', 'type': 'region'}",4.832240e+06,4.832240e+06,10,CalcFBHourglassForceForElems
